In [ ]:
from flask import Flask, request, jsonify
from elasticsearch import Elasticsearch
import nltk
from nltk.tokenize import word_tokenize

# Initialize Flask app
app = Flask(run.py)

# Initialize Elasticsearch client
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Ensure nltk data is downloaded (you may need to install it manually)
nltk.download('punkt')

# Function to index documentation in Elasticsearch
def index_documentation():
    docs = [
        {"platform": "Segment", "title": "Setting up a new source", "content": "To set up a new source in Segment, follow these steps..."},
        {"platform": "mParticle", "title": "Creating a user profile", "content": "To create a user profile in mParticle, follow these steps..."},
        {"platform": "Lytics", "title": "Building an audience segment", "content": "To build an audience segment in Lytics, follow these steps..."},
        {"platform": "Zeotap", "title": "Integrating your data", "content": "To integrate data with Zeotap, follow these steps..."}
    ]
    
    # Index documents in Elasticsearch
    for doc in docs:
        es.index(index="cdp_docs", document=doc)

# Function to search Elasticsearch based on user's query
def search_documentation(query):
    search_response = es.search(
        index="cdp_docs",
        body={
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["title", "content"]
                }
            }
        }
    )
    
    # If no results found, return a generic response
    if search_response['hits']['total']['value'] == 0:
        return "Sorry, I couldn't find an answer for your question."
    
    # Get the best match
    best_match = search_response['hits']['hits'][0]['_source']
    return best_match['content']

# Route to process user questions
@app.route('/ask', methods=['POST'])
def ask():
    # Get the question from the user
    user_question = request.json['question']
    
    # Tokenize the question (for NLP processing)
    tokens = word_tokenize(user_question.lower())
    
    # Define some basic responses for unsupported questions
    unsupported_questions = ["movie", "weather", "news"]
    
    # If the question is unsupported, respond accordingly
    if any(word in tokens for word in unsupported_questions):
        return jsonify({"answer": "Sorry, I can only answer questions related to CDPs."})
    
    # Search for relevant documentation
    answer = search_documentation(user_question)
    
    return jsonify({"answer": answer})

# Route to handle server start
@app.route('/index_docs', methods=['GET'])
def index_docs():
    # Initialize Elasticsearch index if it doesn't exist
    if not es.indices.exists(index="cdp_docs"):
        index_documentation()
    return jsonify({"message": "Documentation indexed successfully!"})

if __name__ == '__main__':
    # Start the Flask server
    app.run(debug=True)
